In [45]:
import os
import pandas as pd
import numpy as np
from math import *
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import *
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday

import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
path_10q = "10Q"
working_path = "/Users/yongzhongxu/Desktop/SIM-Analysis"
os.chdir(working_path)

ticker="mmc"
cik_df = pd.read_csv("tickers.csv")
keys = cik_df["ticker"].values
values = cik_df["cik"].values.astype("str")
cik_dict = dict(zip(keys, values))
os.chdir(path_10q)
cik = cik_dict[ticker] 
m = len(cik)
os.chdir(cik)
# Get list of scraped files
# excluding hidden files and directories
file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname))]
dates_str= [fname[(m+1):(m+11)] for fname in file_list]
dates = [pd.to_datetime(fname[(m+1):(m+11)]).date() for fname in file_list]
dates.sort()
file = "metrics/"+cik+"_sim_scores.csv"
sim = pd.read_csv(file, header = None, skiprows = 4)
os.chdir("../..")

In [39]:
sim = sim.rename(columns={0: "Date", 1:"cos_sim", 2:"jaccard_sim"}).set_index("Date")

In [40]:
sim.ix[dates_str[10]]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


cos_sim        0.786502
jaccard_sim    0.647875
Name: 2006-05-10, dtype: float64

In [41]:
# get NYSE holidays #
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay

class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]

def get_trading_close_holidays(year):
    inst = USTradingCalendar()
    return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))

if __name__ == '__main__':
    print(get_trading_close_holidays(2016))

DatetimeIndex(['2016-01-01', '2016-01-18', '2016-02-15', '2016-03-25',
               '2016-05-30', '2016-07-04', '2016-09-05', '2016-11-24',
               '2016-12-26'],
              dtype='datetime64[ns]', freq=None)


In [42]:
for d in dates:
    y = d.year
    print(d, d in get_trading_close_holidays(y) )


2001-05-15 False
2001-08-13 False
2001-11-14 False
2002-05-14 False
2002-08-13 False
2002-11-14 False
2003-05-13 False
2003-08-14 False
2003-11-14 False
2004-05-10 False
2004-08-03 False
2004-11-09 False
2005-05-06 False
2005-08-09 False
2005-11-09 False
2006-05-10 False
2006-08-09 False
2006-11-09 False
2007-05-10 False
2007-08-09 False
2007-11-09 False
2008-05-08 False
2008-08-08 False
2008-11-07 False
2009-05-08 False
2009-08-07 False
2009-11-05 False
2010-05-07 False
2010-08-06 False
2010-11-09 False
2011-05-06 False
2011-08-05 False
2011-11-04 False
2012-05-08 False
2012-08-08 False
2012-11-08 False
2013-05-09 False
2013-08-08 False
2013-11-07 False
2014-05-08 False
2014-08-04 False
2014-11-04 False
2015-05-04 False
2015-07-31 False
2015-10-29 False
2016-05-02 False
2016-08-01 False
2016-10-28 False
2017-04-28 False
2017-07-28 False
2017-10-27 False
2018-04-27 False
2018-07-27 False
2018-10-26 False
2019-04-26 False


In [49]:
mmc= pd.read_csv("MMC_full.csv",  header=None, skiprows=2)
mmc = mmc.rename(columns={0: "Date", 5:"Adj_close"}).set_index("Date")
mmc = mmc[["Adj_close"]]


,Adj_close
Date,
2000-01-03,27.494371
2000-01-04,26.258673
2000-01-05,25.313740
2000-01-06,27.157269
2000-01-07,28.020283


In [115]:
mmc["5d"]= np.round(pd.rolling_mean(mmc['Adj_close'], window=5), 2)
mmc.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
  """Entry point for launching an IPython kernel.


,Adj_close,5d
Date,,
2000-01-03,27.494371,NaN
2000-01-04,26.258673,NaN
2000-01-05,25.313740,NaN
2000-01-06,27.157269,NaN
2000-01-07,28.020283,26.85


In [116]:
n = len(dates)
logret1 = np.zeros(n-3)
logret2= np.zeros(n-3)
logret3= np.zeros(n-3)
logret4 = np.zeros(n-3)
logret5= np.zeros(n-3)
logret6= np.zeros(n-3)
for k in range(2, n-1):
    d = dates[k]
    d0 = d + relativedelta(days=-1)
    while (d0 in get_trading_close_holidays(y) or  d0.weekday()>4  ):
        d0 = d0 + relativedelta(days=-1)
    dd0 = d0.strftime("%Y-%m-%d")
    y = d.year
    d1 = d + relativedelta(months=+1)
    while (d1 in get_trading_close_holidays(y) or  d1.weekday()>4 ):
        d1 = d1+ relativedelta(days=-1)
    dd1 = d1.strftime("%Y-%m-%d")
    d2 = d + relativedelta(months=+2)
    while (d2 in get_trading_close_holidays(y) or  d2.weekday()>4):
        d2 = d2 + relativedelta(days=-1)
    dd2 = d2.strftime("%Y-%m-%d")
    d3 = d + relativedelta(months=+3)
    while (d3 in get_trading_close_holidays(y) or d3.weekday()>4 ):
        d3 = d3 + relativedelta(days=-1)
    dd3 = d3.strftime("%Y-%m-%d")
    print(d, d0, d1, dd2, dd3)
    logret1[k-2] = log(mmc.ix[dd1]['Adj_close']/mmc.ix[dd0]['Adj_close'])
    logret2[k-2] = log(mmc.ix[dd2]['Adj_close']/mmc.ix[dd0]['Adj_close'])
    logret3[k-2] = log(mmc.ix[dd3]['Adj_close']/mmc.ix[dd0]['Adj_close'])
    
    logret4[k-2] = log(mmc.ix[dd1]['5d']/mmc.ix[dd0]['5d'])
    logret5[k-2] = log(mmc.ix[dd2]['5d']/mmc.ix[dd0]['5d'])
    logret6[k-2] = log(mmc.ix[dd3]['5d']/mmc.ix[dd0]['5d'])
    print(d, logret1[k-2], logret2[k-2])

2001-11-14 2001-11-13 2001-12-14 2002-01-14 2002-02-14
2001-11-14 -0.016792475447569323 -0.003118090085016538
2002-05-14 2002-05-13 2002-06-14 2002-07-12 2002-08-14
2002-05-14 -0.08836082119445265 -0.1284451666082928
2002-08-13 2002-08-12 2002-09-13 2002-10-11 2002-11-13
2002-08-13 0.011233590830252218 -0.06416611682332077
2002-11-14 2002-11-13 2002-12-13 2003-01-14 2003-02-14
2002-11-14 -0.01848605368916326 0.04732035567233573
2003-05-13 2003-05-12 2003-06-13 2003-07-11 2003-08-13
2003-05-13 0.10358133893626995 0.0784614632637367
2003-08-14 2003-08-13 2003-09-12 2003-10-14 2003-11-14
2003-08-14 -0.025135159643995254 -0.042574031738566054
2003-11-14 2003-11-13 2003-12-12 2004-01-14 2004-02-13
2003-11-14 -0.0013220477561604511 0.04639073897901107
2004-05-10 2004-05-07 2004-06-10 2004-07-09 2004-08-10
2004-05-10 -0.009575926448326345 0.021267009035495433
2004-08-03 2004-08-02 2004-09-03 2004-10-01 2004-11-03
2004-08-03 0.03213104910326283 0.04997705575467034
2004-11-09 2004-11-08 2004-12

In [119]:
mmc_cor1 = np.corrcoef(sim["cos_sim"][:51], logret4[1:])[0, 1]
mmc_cor1

-0.051619977393729734

In [114]:
sim["cos_sim"][:51]

Date
2002-05-14    0.764594
2002-08-13    0.755765
2002-11-14    0.741068
2003-05-13    0.755956
2003-08-14    0.760084
2003-11-14    0.723737
2004-05-10    0.766585
2004-08-03    0.745914
2004-11-09    0.711866
2005-05-06    0.705503
2005-08-09    0.705641
2005-11-09    0.728801
2006-05-10    0.786502
2006-08-09    0.789204
2006-11-09    0.799274
2007-05-10    0.803430
2007-08-09    0.785518
2007-11-09    0.780095
2008-05-08    0.764377
2008-08-08    0.770230
2008-11-07    0.768517
2009-05-08    0.833998
2009-08-07    0.810126
2009-11-05    0.790281
2010-05-07    0.809711
2010-08-06    0.804440
2010-11-09    0.825618
2011-05-06    0.823134
2011-08-05    0.827326
2011-11-04    0.753729
2012-05-08    0.783979
2012-08-08    0.786459
2012-11-08    0.817521
2013-05-09    0.820503
2013-08-08    0.838941
2013-11-07    0.838102
2014-05-08    0.825547
2014-08-04    0.823423
2014-11-04    0.821211
2015-05-04    0.817322
2015-07-31    0.823433
2015-10-29    0.828517
2016-05-02    0.811688
2016-0

In [109]:
len(sim["cos_sim"][:51])

51

In [83]:
dates

[datetime.date(2001, 5, 15),
 datetime.date(2001, 8, 13),
 datetime.date(2001, 11, 14),
 datetime.date(2002, 5, 14),
 datetime.date(2002, 8, 13),
 datetime.date(2002, 11, 14),
 datetime.date(2003, 5, 13),
 datetime.date(2003, 8, 14),
 datetime.date(2003, 11, 14),
 datetime.date(2004, 5, 10),
 datetime.date(2004, 8, 3),
 datetime.date(2004, 11, 9),
 datetime.date(2005, 5, 6),
 datetime.date(2005, 8, 9),
 datetime.date(2005, 11, 9),
 datetime.date(2006, 5, 10),
 datetime.date(2006, 8, 9),
 datetime.date(2006, 11, 9),
 datetime.date(2007, 5, 10),
 datetime.date(2007, 8, 9),
 datetime.date(2007, 11, 9),
 datetime.date(2008, 5, 8),
 datetime.date(2008, 8, 8),
 datetime.date(2008, 11, 7),
 datetime.date(2009, 5, 8),
 datetime.date(2009, 8, 7),
 datetime.date(2009, 11, 5),
 datetime.date(2010, 5, 7),
 datetime.date(2010, 8, 6),
 datetime.date(2010, 11, 9),
 datetime.date(2011, 5, 6),
 datetime.date(2011, 8, 5),
 datetime.date(2011, 11, 4),
 datetime.date(2012, 5, 8),
 datetime.date(2012, 8, 8